# MlFlow Experiment Tracking Setup

In [ ]:
import mlflow
mlflow.autolog()
mlflow.set_tracking_uri("http://localhost:8080")
mlflow.set_experiment("Initial Model Evaluation")
mlflow.set_tag("mlflow.runName", "Original_Model")
mlflow.set_experiment_tag

# Imports

In [ ]:
import os
import cv2
import sys
from PIL import Image
from torchvision.transforms.functional import to_pil_image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, random_split, Subset
#Using Resnet50 for classification
import torchvision.models as models
import torch.nn as nn
from torchmetrics import Accuracy, Precision, Recall, F1Score
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
import torch.optim as optim
import numpy as np

MAIN_DIR = Path("__file__").resolve().parent.parent.parent
CLASSIFICATION_MAPPING_DIR = os.path.join(MAIN_DIR, "classification_mapping") 
if CLASSIFICATION_MAPPING_DIR not in sys.path:
    sys.path.insert(0, CLASSIFICATION_MAPPING_DIR)
from classification_mapping import CLASSIFICATION_MAPPING

DATA_DIR = os.path.join(MAIN_DIR, 'data')
TRANSLATED_DATA_DIR = os.path.join(MAIN_DIR, 'data_original_model_translated')

In [ ]:
DEVICE = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {DEVICE} device")

# Loading Dataset

In [ ]:
# Function to transform images to 224x224 for ResNet with normalization based on
# original model image normalization settings
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

In [ ]:
data, labels = [], []

for dir_ in os.listdir(DATA_DIR):
    path = os.path.join(DATA_DIR, dir_)
    translated_path = os.path.join(TRANSLATED_DATA_DIR, dir_)
    if not os.path.exists(translated_path): os.makedirs(translated_path)

    for img in os.listdir(path):
        # Load and transform the image
        image_path = os.path.join(path, img)
        image = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        image_tensor = transform(img_pil)

        data.append(image_tensor)
        labels.append(dir_)
        
        image_pil = to_pil_image(image_tensor)
        image_translated_path = os.path.join(translated_path, img)
        image_pil.save(image_translated_path)

In [ ]:
print(len(data))

In [ ]:
class SignDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = torch.tensor([CLASSIFICATION_MAPPING[label] for label in labels], dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

    @property
    def classes(self):
        return self.data.classes

In [ ]:
dataset = SignDataset(data, labels)
len(dataset)

In [ ]:
# Stratified K Folde Cross Validation
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Function to Create CNN Model from Pytorch

In [ ]:
num_classes = 4

def create_model():
    return CNN()
    
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.net = nn.Sequential(
            # Convolutional layers
            nn.Conv2d(3, 16, kernel_size=3, padding=1),   # -> (16, 512, 512)
            nn.ReLU(),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # -> (32, 512, 512)
            nn.ReLU(),
            nn.MaxPool2d(2),                              # -> (32, 216, 216)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # -> (64, 216, 216)
            nn.ReLU(),

            nn.Conv2d(64, 128, kernel_size=3, padding=1), # -> (128, 216, 216)
            nn.ReLU(),
            nn.MaxPool2d(2),                              # -> (128, 128, 128)

            nn.Conv2d(128, 256, kernel_size=3, padding=1),# -> (256, 128, 128)
            nn.ReLU(),

            nn.Flatten(),                                 # -> 256 * 128 * 128 = 4,194,304

            # Fully connected layers with dropout
            nn.Linear(256 * 128 * 128, 512),
            nn.ReLU(),
            nn.Linear(256 * 128 * 128, 512),
            nn.Dropout(p=0.5),  # Dropout applied after first FC ReLU

            nn.Linear(512, num_classes),
            nn.LogSoftmax(dim=1),
        )

    def forward(self, x):
        return self.net(x)

# Grid Search Parameters

In [ ]:
# Grid Search Parameters
param_grid = {
    'lr': [0.01, 0.001, 0.0001],
    'train_batch_size': [16, 32, 64],
    'test_batch_size': [16],
    'weight_decay': [0.0, 1e-4],
}

In [ ]:
# Use itertools.product to iterate through combinations
from itertools import product

param_combinations = list(product(*param_grid.values()))
param_names = list(param_grid.keys())

# Model training

In [ ]:
def train(model, optimizer, dataloader, criterion, device):
    model.train()
    for data, labels in dataloader:
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [ ]:
def evaluate(model, dataloader, device):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for data, labels in dataloader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

In [ ]:
categories = []
accuracies = []
results = []
num_epochs = 40
for params in param_combinations:
    config = dict(zip(param_names, params))
    print(f"Testing params: {config}")
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(data, labels)):
        print(f"Fold {fold+1}")
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        train_loader = torch.utils.data.DataLoader(train_subset, batch_size=config['train_batch_size'], shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_subset, batch_size=config['test_batch_size'])

        model = create_model()  # Initialize fresh model per fold
        optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
        criterion = torch.nn.CrossEntropyLoss()
        model.to(DEVICE)

        # Train and evaluate
        for epoch in range(num_epochs):  # or another stopping criterion
            train(model, optimizer, train_loader, criterion, DEVICE)

        acc = evaluate(model, val_loader, DEVICE)
        fold_accuracies.append(acc)

    avg_acc = np.mean(fold_accuracies)
    print(f"Average Accuracy: {avg_acc:.4f}")
    results.append((config, avg_acc))
    categories.append(str(config).replace("'", ""))
    accuracies.append(avg_acc)

# Visualize

In [ ]:
plt.barh(categories, accuracies)
plt.legend()
plt.title("Accuracy Per Grid Search Per 5 K fold/40 epochs")
plt.show()

# Model Evaluation Logging

In [ ]:
# Log result
for category, accuracy in zip(categories, accuracies):
    mlflow.log_metric(category.replace("'", "").replace("{","").replace("}","").replace(": ", "").replace(", ", " "), accuracy)